In [ ]:
import json
import uuid
import datetime
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, asdict
from abc import ABC, abstractmethod
import asyncio
import schedule
import time
import threading
from enum import Enum
import logging

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
class ConversationStage(Enum):
    INITIAL_GREETING = "initial_greeting"
    COLLECTING_BASIC_INFO = "collecting_basic_info"
    SELF_INTRODUCTION = "self_introduction"
    ACADEMIC_PROFILE_BUILDING = "academic_profile_building"
    PROFILE_COMPLETE = "profile_complete"
    DAILY_INTERACTION = "daily_interaction"

In [ ]:
@dataclass
class StudentProfile:
    student_id: str
    name: str
    nationality: str = ""
    gender: str = ""
    age: int = 0
    graduation_year: int = 0
    high_school: str = ""
    
    # Academic Background
    current_gpa: float = 0.0
    target_gpa: float = 0.0
    standardized_tests: Dict[str, Any] = None
    target_tests: Dict[str, Any] = None
    current_courses: List[str] = None
    target_courses: List[str] = None
    
    # Competitions and Activities
    competitions: List[Dict] = None
    target_competitions: List[Dict] = None
    activities: List[Dict] = None
    target_activities: List[Dict] = None
    
    # Goals and Recommendations
    target_major: str = ""
    target_schools: List[str] = None
    personal_background: str = ""
    personality_traits: List[str] = None
    
    # Conversation tracking
    conversation_stage: str = ConversationStage.INITIAL_GREETING.value
    last_interaction: datetime.datetime = None
    
    def __post_init__(self):
        if self.standardized_tests is None:
            self.standardized_tests = {}
        if self.target_tests is None:
            self.target_tests = {}
        if self.current_courses is None:
            self.current_courses = []
        if self.target_courses is None:
            self.target_courses = []
        if self.competitions is None:
            self.competitions = []
        if self.target_competitions is None:
            self.target_competitions = []
        if self.activities is None:
            self.activities = []
        if self.target_activities is None:
            self.target_activities = []
        if self.target_schools is None:
            self.target_schools = []
        if self.personality_traits is None:
            self.personality_traits = []
        if self.last_interaction is None:
            self.last_interaction = datetime.datetime.now()

In [ ]:
class RAGSystem:
    """Simple RAG system for academic knowledge"""
    
    def __init__(self):
        self.knowledge_base = {
            "universities": {
                "ivy_league": ["Harvard", "Yale", "Princeton", "Columbia", "Penn", "Dartmouth", "Brown", "Cornell"],
                "top_engineering": ["MIT", "Stanford", "Caltech", "CMU", "UC Berkeley"],
                "business_schools": ["Wharton", "Sloan", "Booth", "Kellogg", "Stanford GSB"]
            },
            "standardized_tests": {
                "SAT": {"max_score": 1600, "sections": ["Math", "Reading/Writing"]},
                "ACT": {"max_score": 36, "sections": ["English", "Math", "Reading", "Science"]},
                "TOEFL": {"max_score": 120, "min_requirement": 80},
                "IELTS": {"max_score": 9.0, "min_requirement": 6.5}
            },
            "majors": {
                "STEM": ["Computer Science", "Engineering", "Mathematics", "Physics", "Biology"],
                "Business": ["Finance", "Marketing", "Management", "Economics"],
                "Liberal Arts": ["English", "History", "Philosophy", "Psychology"]
            },
            "application_timeline": {
                "junior_year": ["Take SAT/ACT", "AP exams", "Build activity list"],
                "senior_year": ["Early applications (Nov 1)", "Regular applications (Jan 1)", "FAFSA"]
            }
        }
    
    def search(self, query: str) -> Dict[str, Any]:
        """Simple keyword-based search"""
        results = {}
        query_lower = query.lower()
        
        for category, data in self.knowledge_base.items():
            if any(keyword in query_lower for keyword in category.split('_')):
                results[category] = data
        
        return results

In [ ]:
class CogniGuideAgent:
    def __init__(self):
        self.students: Dict[str, StudentProfile] = {}
        self.rag_system = RAGSystem()
        self.conversation_history: Dict[str, List[Dict]] = {}
        self.daily_schedules: Dict[str, List[str]] = {}
        self.current_questions = {
            "basic_info": [
                "What's your nationality? 🌍",
                "What's your gender?",
                "How old are you?",
                "What year will you graduate from high school?",
                "What high school are you currently attending?"
            ],
            "academic_profile": [
                "What's your current GPA? 📊",
                "Have you taken any standardized tests like SAT, ACT, TOEFL, or IELTS?",
                "What courses are you currently taking? (AP, IB, A-levels, etc.)",
                "Have you participated in any academic competitions?",
                "What extracurricular activities are you involved in?",
                "What major are you interested in pursuing?",
                "Do you have any dream schools in mind?",
                "Tell me about your personal background and what makes you unique! ✨"
            ]
        }
        
        # Start scheduler in background
        self.start_scheduler()
    
    def generate_student_id(self) -> str:
        return f"STU_{uuid.uuid4().hex[:8].upper()}"
    
    def get_or_create_student(self, user_input: str) -> tuple[StudentProfile, bool]:
        """Returns (student_profile, is_new_student)"""
        # Simple logic to detect if user mentions existing ID
        words = user_input.split()
        for word in words:
            if word.startswith("STU_") and word in self.students:
                return self.students[word], False
        
        # Create new student
        student_id = self.generate_student_id()
        student = StudentProfile(student_id=student_id, name="")
        self.students[student_id] = student
        self.conversation_history[student_id] = []
        return student, True
    
    def add_to_history(self, student_id: str, role: str, message: str):
        if student_id not in self.conversation_history:
            self.conversation_history[student_id] = []
        self.conversation_history[student_id].append({
            "role": role,
            "message": message,
            "timestamp": datetime.datetime.now().isoformat()
        })
    
    def process_user_input(self, user_input: str) -> str:
        student, is_new = self.get_or_create_student(user_input)
        student.last_interaction = datetime.datetime.now()
        
        self.add_to_history(student.student_id, "user", user_input)
        
        if is_new:
            response = self.handle_new_student(student)
        else:
            response = self.handle_existing_student(student, user_input)
        
        self.add_to_history(student.student_id, "assistant", response)
        return response
    
    def handle_new_student(self, student: StudentProfile) -> str:
        student.conversation_stage = ConversationStage.INITIAL_GREETING.value
        return f"""Hello there! 👋 Welcome to CogniGuide! I'm your friendly study abroad mentor, and I'm super excited to help you achieve your academic dreams! 🌟

I've generated your unique student ID: **{student.student_id}** - please save this for future conversations!

Before we dive into the fun stuff, I'd love to get to know you better! What's your name? 😊"""
    
    def handle_existing_student(self, student: StudentProfile, user_input: str) -> str:
        stage = ConversationStage(student.conversation_stage)
        
        if stage == ConversationStage.INITIAL_GREETING:
            return self.collect_name(student, user_input)
        elif stage == ConversationStage.COLLECTING_BASIC_INFO:
            return self.collect_basic_info(student, user_input)
        elif stage == ConversationStage.SELF_INTRODUCTION:
            return self.handle_self_introduction(student, user_input)
        elif stage == ConversationStage.ACADEMIC_PROFILE_BUILDING:
            return self.build_academic_profile(student, user_input)
        elif stage == ConversationStage.PROFILE_COMPLETE:
            return self.handle_daily_interactions(student, user_input)
        else:
            return self.handle_daily_interactions(student, user_input)
    
    def collect_name(self, student: StudentProfile, user_input: str) -> str:
        student.name = user_input.strip()
        student.conversation_stage = ConversationStage.COLLECTING_BASIC_INFO.value
        return f"""Nice to meet you, {student.name}! 🤗 

Now, let's gather some basic information about you. Don't worry, this will be quick and painless! 

{self.current_questions["basic_info"][0]}"""
    
    def collect_basic_info(self, student: StudentProfile, user_input: str) -> str:
        # Simple state machine for collecting basic info
        if not student.nationality:
            student.nationality = user_input.strip()
            return f"Got it! {student.nationality} - that's awesome! 🌍\n\n{self.current_questions['basic_info'][1]}"
        elif not student.gender:
            student.gender = user_input.strip()
            return f"Thanks for sharing! \n\n{self.current_questions['basic_info'][2]}"
        elif student.age == 0:
            try:
                student.age = int(user_input.strip())
                return f"Perfect! {student.age} is a great age to start planning! 🎯\n\n{self.current_questions['basic_info'][3]}"
            except ValueError:
                return "Oops! Please enter your age as a number 😊"
        elif student.graduation_year == 0:
            try:
                student.graduation_year = int(user_input.strip())
                return f"Excellent! Class of {student.graduation_year}! 🎓\n\n{self.current_questions['basic_info'][4]}"
            except ValueError:
                return "Please enter your graduation year as a number (e.g., 2025) 📅"
        elif not student.high_school:
            student.high_school = user_input.strip()
            student.conversation_stage = ConversationStage.SELF_INTRODUCTION.value
            return f"""Wonderful! {student.high_school} sounds like a great school! 🏫

Now that I know the basics, I'd love to hear more about YOU! Could you give me a brief self-introduction? Tell me about:

✨ What you're passionate about
🎯 Your dreams and aspirations  
🌟 What makes you unique
🎨 Your hobbies and interests

Don't be shy - I want to get to know the real {student.name}! 😊"""
    
    def handle_self_introduction(self, student: StudentProfile, user_input: str) -> str:
        student.personal_background = user_input.strip()
        student.conversation_stage = ConversationStage.ACADEMIC_PROFILE_BUILDING.value
        
        return f"""Thank you for sharing, {student.name}! 🌟 I can already tell you're someone special with amazing potential!

Now, let's dive into building your academic profile. This will help me create a personalized roadmap for your success! 

{self.current_questions["academic_profile"][0]}"""
    
    def build_academic_profile(self, student: StudentProfile, user_input: str) -> str:
        # Track which academic info we've collected
        if student.current_gpa == 0.0:
            try:
                gpa = float(user_input.strip())
                student.current_gpa = gpa
                # Set target GPA
                student.target_gpa = min(4.0, gpa + 0.3)
                return f"Great! Current GPA: {student.current_gpa} 📊\nI think we can aim for {student.target_gpa} as your target!\n\n{self.current_questions['academic_profile'][1]}"
            except ValueError:
                return "Please enter your GPA as a number (e.g., 3.5) 📊"
        
        elif not student.standardized_tests:
            if "none" in user_input.lower() or "no" in user_input.lower():
                student.standardized_tests = {"status": "not_taken"}
                student.target_tests = {"SAT": 1450, "TOEFL": 100}
            else:
                student.standardized_tests = {"status": "taken", "details": user_input}
                student.target_tests = {"improvement_needed": True}
            return f"Thanks for the info! 📝\n\n{self.current_questions['academic_profile'][2]}"
        
        elif not student.current_courses:
            student.current_courses = [course.strip() for course in user_input.split(',')]
            student.target_courses = student.current_courses + ["Additional AP courses recommended"]
            return f"Excellent course selection! 📚\n\n{self.current_questions['academic_profile'][3]}"
        
        elif not student.competitions:
            if "none" in user_input.lower() or "no" in user_input.lower():
                student.competitions = [{"status": "none"}]
                student.target_competitions = [{"recommendation": "Math Olympiad or Science Fair"}]
            else:
                student.competitions = [{"details": user_input}]
                student.target_competitions = [{"continue": "current competitions", "add": "national level"}]
            return f"Good to know! Competition experience is valuable! 🏆\n\n{self.current_questions['academic_profile'][4]}"
        
        elif not student.activities:
            student.activities = [activity.strip() for activity in user_input.split(',')]
            student.target_activities = student.activities + ["Leadership role recommended"]
            return f"Awesome activities! 🎭\n\n{self.current_questions['academic_profile'][5]}"
        
        elif not student.target_major:
            student.target_major = user_input.strip()
            # Get recommendations from RAG
            major_info = self.rag_system.search(student.target_major)
            return f"Fantastic choice! {student.target_major} is an exciting field! 🎯\n\n{self.current_questions['academic_profile'][6]}"
        
        elif not student.target_schools:
            student.target_schools = [school.strip() for school in user_input.split(',')]
            return f"Amazing schools! Great choices! 🏛️\n\n{self.current_questions['academic_profile'][7]}"
        
        else:
            # Final step - complete profile
            student.conversation_stage = ConversationStage.PROFILE_COMPLETE.value
            return self.generate_academic_profile(student)
    
    def generate_academic_profile(self, student: StudentProfile) -> str:
        profile_doc = f"""
🎓 **ACADEMIC PROFILE FOR {student.name.upper()}**
Student ID: {student.student_id}
Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

═══════════════════════════════════════

📋 **BASIC INFORMATION**
• Name: {student.name}
• Nationality: {student.nationality}
• Age: {student.age}
• High School: {student.high_school}
• Graduation Year: {student.graduation_year}

📊 **ACADEMIC PERFORMANCE**
Current Status → Target Goals

GPA: {student.current_gpa} → {student.target_gpa}
Courses: {', '.join(student.current_courses)} → Enhanced with advanced courses
Standardized Tests: {student.standardized_tests} → {student.target_tests}

🏆 **COMPETITIONS & ACTIVITIES**
Current: {student.competitions}
Target: {student.target_competitions}

Activities: {', '.join(student.activities)}
Enhanced Goals: Leadership positions and community impact

🎯 **CAREER PLANNING**
Target Major: {student.target_major}
Dream Schools: {', '.join(student.target_schools)}

🌟 **PERSONAL PROFILE**
{student.personal_background}

💡 **RECOMMENDATIONS**
Based on your profile, I recommend:
1. Focus on improving GPA to {student.target_gpa}
2. Prepare for standardized tests systematically
3. Develop leadership roles in current activities
4. Consider research opportunities in {student.target_major}
5. Start building relationships with teachers for recommendations

═══════════════════════════════════════

Your profile is now complete! 🎉 I'll start providing daily schedules and guidance to help you achieve these goals!
"""
        
        # Save as "PDF" (text format)
        self.save_academic_profile(student.student_id, profile_doc)
        
        return f"""🎉 **CONGRATULATIONS {student.name}!** 

Your comprehensive academic profile has been created and saved! 📋✨

{profile_doc}

From now on, I'll be your daily companion, providing:
• 🌅 Morning study schedules (7 AM)
• 🌙 Evening progress check-ins (9 PM)  
• 📚 24/7 academic support
• 🎯 Goal tracking and adjustments

Ready to start this amazing journey together? Let me know if you have any questions about your profile or if you'd like to see your first daily schedule! 🚀"""

    def save_academic_profile(self, student_id: str, profile_content: str):
        """Save academic profile as text file (simulating PDF generation)"""
        filename = f"academic_profile_{student_id}_{datetime.datetime.now().strftime('%Y%m%d')}.txt"
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(profile_content)
            logger.info(f"Academic profile saved: {filename}")
        except Exception as e:
            logger.error(f"Error saving profile: {e}")
    
    def handle_daily_interactions(self, student: StudentProfile, user_input: str) -> str:
        """Handle ongoing daily interactions"""
        user_lower = user_input.lower()
        
        if any(word in user_lower for word in ['schedule', 'plan', 'today', 'tomorrow']):
            return self.create_daily_schedule(student)
        elif any(word in user_lower for word in ['profile', 'document', 'pdf']):
            return self.generate_academic_profile(student)
        elif any(word in user_lower for word in ['help', 'question', 'advice']):
            return self.provide_academic_advice(student, user_input)
        else:
            return self.casual_conversation(student, user_input)
    
    def create_daily_schedule(self, student: StudentProfile) -> str:
        """Create personalized daily schedule"""
        today = datetime.datetime.now().strftime('%A, %B %d, %Y')
        
        schedule_tasks = []
        
        # GPA improvement tasks
        if student.current_gpa < student.target_gpa:
            schedule_tasks.append("📚 Review and complete homework for all classes (2 hours)")
            schedule_tasks.append("✍️ Spend 30 minutes on weakest subject")
        
        # Test prep
        if student.standardized_tests.get('status') == 'not_taken':
            schedule_tasks.append("📝 SAT/ACT practice - one section (45 minutes)")
        
        # Activities and leadership
        schedule_tasks.append("🌟 Spend 1 hour on extracurricular activities")
        schedule_tasks.append("📖 Read for 30 minutes (academic or personal interest)")
        
        # College prep
        if student.target_schools:
            schedule_tasks.append("🏛️ Research one target university (20 minutes)")
        
        schedule_text = f"""
🌅 **DAILY SCHEDULE FOR {student.name}**
{today}

Here's your personalized plan for today:

{chr(10).join(f'• {task}' for task in schedule_tasks)}

💡 **Bonus Tasks:**
• Connect with a teacher or mentor
• Update your activity log
• Practice a skill related to {student.target_major}

Remember, consistency is key! You've got this! 💪✨

How does this schedule look? Any adjustments needed? 😊
"""
        
        self.daily_schedules[student.student_id] = schedule_tasks
        return schedule_text
    
    def provide_academic_advice(self, student: StudentProfile, question: str) -> str:
        """Provide academic guidance using RAG system"""
        search_results = self.rag_system.search(question)
        
        advice = f"Great question, {student.name}! 🤔\n\n"
        
        if 'universities' in search_results:
            advice += "🏛️ **University Insights:** Based on your interests, here are some recommendations...\n"
        
        if 'standardized_tests' in search_results:
            advice += "📝 **Test Strategy:** Let me share some proven approaches...\n"
        
        advice += f"""
Based on your profile and goals in {student.target_major}, here's my personalized advice:

💡 **Immediate Actions:**
• Focus on maintaining strong grades in current courses
• Start building relationships with teachers in relevant subjects
• Look for research or internship opportunities

🎯 **Long-term Strategy:**
• Develop a signature project or achievement in {student.target_major}
• Consider leadership roles in activities you're passionate about
• Start drafting your personal story for college essays

Need more specific guidance on any of these areas? I'm here 24/7! 😊✨
"""
        return advice
    
    def casual_conversation(self, student: StudentProfile, user_input: str) -> str:
        """Handle casual, friendly conversation"""
        responses = [
            f"That's really interesting, {student.name}! 😊 Tell me more about that!",
            f"I love hearing about your experiences! 🌟 How did that make you feel?",
            f"You're such an interesting person, {student.name}! 💫 Thanks for sharing!",
            f"That's awesome! 🎉 It's great to get to know you better outside of academics too!",
            f"I appreciate you opening up! 💕 These conversations help me understand you better as a person."
        ]
        
        import random
        base_response = random.choice(responses)
        
        return f"""{base_response}

By the way, is there anything academic I can help you with today? Or would you like to see your current progress toward your goals? 📚✨"""

    def morning_check_in(self, student_id: str) -> str:
        """7 AM daily schedule delivery"""
        if student_id not in self.students:
            return ""
        
        student = self.students[student_id]
        return f"""🌅 **Good morning, {student.name}!** 

Hope you're ready for another productive day! Here's your personalized schedule:

{self.create_daily_schedule(student)}

You're doing amazing work toward your goals! Keep it up! 💪✨"""

    def evening_check_in(self, student_id: str) -> str:
        """9 PM progress inquiry"""
        if student_id not in self.students:
            return ""
        
        student = self.students[student_id]
        
        return f"""🌙 **Evening check-in, {student.name}!** 

How did your day go? I'd love to hear about:

✅ Which tasks from today's schedule did you complete?
😅 What challenges did you face?
🎉 Any wins or breakthroughs to celebrate?
🤔 What can we adjust for tomorrow?

Remember, every step forward counts! I'm proud of your dedication! 🌟💕"""

    def start_scheduler(self):
        """Start background scheduler for daily check-ins"""
        def run_scheduler():
            schedule.every().day.at("07:00").do(self.send_morning_schedules)
            schedule.every().day.at("21:00").do(self.send_evening_checkins)
            
            while True:
                schedule.run_pending()
                time.sleep(60)  # Check every minute
        
        scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
        scheduler_thread.start()
    
    def send_morning_schedules(self):
        """Send morning schedules to all active students"""
        for student_id, student in self.students.items():
            if student.conversation_stage == ConversationStage.PROFILE_COMPLETE.value:
                message = self.morning_check_in(student_id)
                self.add_to_history(student_id, "system", message)
                logger.info(f"Morning schedule sent to {student.name}")
    
    def send_evening_checkins(self):
        """Send evening check-ins to all active students"""
        for student_id, student in self.students.items():
            if student.conversation_stage == ConversationStage.PROFILE_COMPLETE.value:
                message = self.evening_check_in(student_id)
                self.add_to_history(student_id, "system", message)
                logger.info(f"Evening check-in sent to {student.name}")

    def save_student_data(self, filename: str = "students_data.json"):
        """Save all student data to file"""
        data = {}
        for student_id, student in self.students.items():
            student_dict = asdict(student)
            student_dict['last_interaction'] = student.last_interaction.isoformat()
            data[student_id] = student_dict
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            logger.info(f"Student data saved to {filename}")
        except Exception as e:
            logger.error(f"Error saving student data: {e}")
    
    def load_student_data(self, filename: str = "students_data.json"):
        """Load student data from file"""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            for student_id, student_dict in data.items():
                student_dict['last_interaction'] = datetime.datetime.fromisoformat(student_dict['last_interaction'])
                self.students[student_id] = StudentProfile(**student_dict)
                if student_id not in self.conversation_history:
                    self.conversation_history[student_id] = []
            
            logger.info(f"Student data loaded from {filename}")
        except FileNotFoundError:
            logger.info("No existing student data file found")
        except Exception as e:
            logger.error(f"Error loading student data: {e}")

In [ ]:
# Demo/Testing Interface
class CogniGuideChat:
    def __init__(self):
        self.agent = CogniGuideAgent()
        self.agent.load_student_data()
    
    def start_chat(self):
        print("🎓 Welcome to CogniGuide! Your AI Study Abroad Mentor 🌟")
        print("Type 'quit' to exit, 'save' to save data")
        print("="*60)
        
        while True:
            user_input = input("\n👤 You: ").strip()
            
            if user_input.lower() == 'quit':
                self.agent.save_student_data()
                print("👋 Goodbye! Keep chasing your dreams! ✨")
                break
            elif user_input.lower() == 'save':
                self.agent.save_student_data()
                print("💾 Data saved successfully!")
                continue
            elif user_input.lower() == 'students':
                print(f"📋 Active students: {len(self.agent.students)}")
                for sid, student in self.agent.students.items():
                    print(f"  • {student.name} ({sid}) - Stage: {student.conversation_stage}")
                continue
            
            if user_input:
                response = self.agent.process_user_input(user_input)
                print(f"\n🤖 CogniGuide: {response}")

In [ ]:
# Main execution
if __name__ == "__main__":
    # Run the chat interface
    chat = CogniGuideChat()
    chat.start_chat()